# Exploring Topic and Word Relationships

Here we explore different ways that the the newstest 2015 dataset can be looked at.

In [2]:
import math

def read_file(file_path, max_amount=math.inf):
    """
    Reads a number of lines from file and return list of lines
    :param file_path:
    :param max_amount:
    :return: list[str]
    """
    line_list = []

    with open(file=file_path) as f:
        curr_idx = 0
        while curr_idx < max_amount:
            line_list.append(f.readline())
            curr_idx += 1

    return line_list

print("Loading file")
l_raw = read_file('./trg.shuf', 5000000)
print("Finished loading in file")

Loading file
Finished loading in file


In [20]:
import math
import pandas as pd
from collections import Counter
import operator
import nltk
nltk.download('wordnet')
from nltk.corpus import wordnet as wn
from nltk.stem.wordnet import WordNetLemmatizer
nltk.download('stopwords')
en_stop = set(nltk.corpus.stopwords.words('english'))
import gensim
from gensim import corpora



def process_word(word, remove_stop=False, lemmatize_and_morph=True):
    """
    Returns process word
    :param word:
    :return: str word
    """
    # lower
    word = word.lower()
    
    # Remove symbols
    if len(word) < 3:
        return None
    
    # Get morph
    if lemmatize_and_morph is True:
        t_word = wn.morphy(word)
        if t_word is not None:
            word = t_word

        # Get lemma
        word = WordNetLemmatizer().lemmatize(word)
    
    # Remove stop
    if remove_stop is True and word in en_stop:
        return None
    return word


def preprocess_all_words(word_list):
    ret = []
    for line in word_list:
        temp = []
        for word in line:
            w = process_word(word, remove_stop=True, lemmatize_and_morph=False)
            if w is not None:
                temp.append(w)
        ret.append(temp)
    return ret


def process_bpe(line_list, seq="@@", rm_words=["\n"], lemmatize_and_morph=True):
    """
    Transforms line list form read_file to list[list[str=words]], and bpe words merged together
    :param line_list:
    :param seq:
    :param rm_words:
    :return: list[list[str=words]]
    """
    full_word_list = []
    for line in line_list:
        temp_words = line.split()
        full_words = []
        idx = 0
        while idx < len(temp_words):
            if temp_words[idx].endswith(seq):
                temp_str = ""
                while temp_words[idx].endswith(seq):
                    temp_str += temp_words[idx][:-(len(seq))]
                    idx += 1
                temp_str += temp_words[idx]
                if temp_str not in rm_words:
                    w = process_word(temp_str, lemmatize_and_morph=lemmatize_and_morph)
                    if w is not None:
                        full_words.append(w)
                idx += 1
            else:
                if temp_words[idx] not in rm_words:
                    w = process_word(temp_words[idx], lemmatize_and_morph=lemmatize_and_morph)
                    if w is not None:
                        full_words.append(w)
                idx += 1
        full_word_list.append(full_words)
    return full_word_list


def get_word_count(word_list):
    """
    Returns word count.
    :param word_list:
    :return: List[(str=word, int=count)]
    """
    word_count = {}
    for line in word_list:
        for word, val in Counter(line).most_common():
            if word in word_count:
                word_count[word] += val
            else:
                word_count[word] = val

    return sorted(word_count.items(), key=operator.itemgetter(1))


def get_topics_lda(word_list, dictionary, num_topics=10):
    corpus = [dictionary.doc2bow(text) for text in word_list]
    print("Finished corpus")
    
    ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics=num_topics, id2word=dictionary, passes=15)
    topics = ldamodel.print_topics(num_words=4)
    print("Topics: ")
    for topic in topics:
        print(topic)

    return topics, ldamodel, corpus, dictionary

import random
l = random.sample(l_raw, 10000)

wl = process_bpe(line_list=l)
print("Vocab size train: " + str(len(get_word_count(word_list=wl))))

wl_full = process_bpe(line_list=l_raw, lemmatize_and_morph=False)
print("Vocab size full: " + str(len(get_word_count(word_list=wl_full))))

dictionary = corpora.Dictionary(wl_full)
print("Finished Dictionary")

t, ldamodel,_, dictionary = get_topics_lda(word_list=preprocess_all_words(wl), dictionary=dictionary)




[nltk_data] Downloading package wordnet to /home/nikita/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /home/nikita/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Vocab size small: 17870
Vocab size full: 1001244
Finished Dictionary
Finished corpus
Topics: 
(0, '0.000*"august" + 0.000*"tuesday" + 0.000*"2016" + 0.000*"november"')
(1, '0.001*"interest" + 0.000*"visit" + 0.000*"town" + 0.000*"2014"')
(2, '0.003*"say" + 0.003*"one" + 0.002*"also" + 0.002*"time"')
(3, '0.001*"2012" + 0.001*"page" + 0.001*"law" + 0.001*"demand"')
(4, '0.001*"2016" + 0.001*"post" + 0.001*"http" + 0.001*"currently"')
(5, '0.002*"night" + 0.001*"sat" + 0.000*"method" + 0.000*"maintain"')
(6, '0.001*"two" + 0.001*"report" + 0.001*"city" + 0.001*"find"')
(7, '0.001*"2015" + 0.000*"sunday" + 0.000*"march" + 0.000*"thursday"')
(8, '0.005*"amp" + 0.000*"delivery" + 0.000*"south" + 0.000*"http"')
(9, '0.001*"try" + 0.000*"practice" + 0.000*"spend" + 0.000*"safety"')





# Visualization
Here we visualize with pyLDAvis in an interactive mode the LDA analysis.

In [21]:
import pyLDAvis.gensim
from gensim import corpora
dictionary = corpora.Dictionary(wl)
corpus = [dictionary.doc2bow(text) for text in wl]
lda_display = pyLDAvis.gensim.prepare(ldamodel, corpus, dictionary, sort_topics=False)
pyLDAvis.display(lda_display)

ValidationError: 
 * Not all rows (distributions) in topic_term_dists sum to 1.

# Vocab Splitting

Here we get the vocab for each topic.

In [26]:
def get_representatives_lda(word_list, lda_model, dictionary, num_topics=10):
    max_line = [[]] * num_topics
    max_class = [0] * num_topics 
    class_vocab = {i:{} for i in range(0, num_topics)}
    
    for line, idx in zip(word_list, range(len(word_list))):
        #print(line)
        new_doc_bow = dictionary.doc2bow(line)
        s = ldamodel.get_document_topics(new_doc_bow)
        s.sort(key=lambda x: x[1])
        t_max_class, t_max_perc = s[-1]
        
        for word in line:
            if word in class_vocab[t_max_class]:
                class_vocab[t_max_class][word] += 1
            else:
                class_vocab[t_max_class][word] = 1
        
        if max_class[t_max_class] < t_max_perc:
            max_class[t_max_class] = t_max_perc
            max_line[t_max_class] = line
        
        if idx % 1000000 == 0:
            print("Representatives: " + str(idx))
        
    print("Finished representatives")
    return max_line, max_class, class_vocab

# max_l, max_c, class_vocab = get_representatives_lda(wl, ldamodel, dictionary)
"""
for idx in range(len(max_l)):
    print("\n")
    print("\n")
    print(max_c[idx])
    print(max_l[idx])
"""  

# get overlap
num_topics = 10

import numpy as np


def get_overlap_matrix(class_vocab, num_topics):
    over_lap_matrix = np.ones((num_topics, num_topics))

    for c in range(0, num_topics):
        for c2 in range(0, num_topics):
            # Skip self
            if c2 == c:
                continue
            # overlap
            words_in_c2_and_c1 = 0
            total_words_c2 = 0
            for word in class_vocab[c2]:
                total_words_c2 += 1  # class_vocab[c2][word]
                if word in class_vocab[c]:
                    words_in_c2_and_c1 += 1
            c_factor = len(class_vocab[c])/(len(class_vocab[c2]) + len(class_vocab[c]))
            c2_factor = len(class_vocab[c2])/(len(class_vocab[c2]) + len(class_vocab[c]))
            over_lap_matrix[c, c2] = (words_in_c2_and_c1/len(class_vocab[c2])) * c2_factor + (words_in_c2_and_c1/len(class_vocab[c])) * c_factor
    
    return over_lap_matrix


over_lap_matrix = get_overlap_matrix(get_representatives_lda(wl_full, ldamodel, dictionary, num_topics=num_topics)[2], num_topics)

print("Overlap Matrix of Vocab")
print(pd.DataFrame(over_lap_matrix))
print("\n")

print("Average for each topic")
# Remove diagonal as it skews data
print(pd.DataFrame(np.average(over_lap_matrix[~np.eye(over_lap_matrix.shape[0],dtype=bool)].reshape(over_lap_matrix.shape[0],-1), axis=1)))
print("\n")

print("Average overall")
print(np.average(over_lap_matrix[~np.eye(over_lap_matrix.shape[0],dtype=bool)].reshape(over_lap_matrix.shape[0],-1)))

print("\n")
print("Max")
print(np.max(over_lap_matrix[~np.eye(over_lap_matrix.shape[0],dtype=bool)].reshape(over_lap_matrix.shape[0],-1)))
                

Representatives: 0
Representatives: 1000000
Representatives: 2000000
Representatives: 3000000
Representatives: 4000000
Finished representatives
Overlap Matrix of Vocab
          0         1         2         3         4         5         6  \
0  1.000000  0.433109  0.296577  0.377850  0.430533  0.419403  0.396045   
1  0.433109  1.000000  0.257183  0.369511  0.434925  0.433718  0.370654   
2  0.296577  0.257183  1.000000  0.275627  0.249150  0.231479  0.366931   
3  0.377850  0.369511  0.275627  1.000000  0.363572  0.352985  0.356752   
4  0.430533  0.434925  0.249150  0.363572  1.000000  0.429736  0.359918   
5  0.419403  0.433718  0.231479  0.352985  0.429736  1.000000  0.345667   
6  0.396045  0.370654  0.366931  0.356752  0.359918  0.345667  1.000000   
7  0.441388  0.441597  0.276756  0.374693  0.437972  0.428611  0.387070   
8  0.409641  0.430209  0.212272  0.340930  0.430837  0.432849  0.325636   
9  0.382166  0.373526  0.296171  0.338647  0.364056  0.352732  0.364138   

      